import pygame
from tkinter import *

pygame.init()
mixer.music.load('filename.wav')

class Track:
    """
    The "Track" object holds 
    """
    def __init__(self, title = "N/A", artist = "N/A", file = "error.wav"): # default values set
    self.title = title
    self.year = year
    self.file = file
    
    @property
    def title(self):
        """
        Function to return the track's title.
        """
        return self.__title
    
    @property
    def artist(self):
        """
        Function to return the track's artist.
        """
        return self.__artist
    
    @property
    def file(self):
        """
        Function to return the track's file.
        """
        return self.__file
    

def playTrack():
    """
    Function to play and resume a track.
    """
    mixer.music.play() # play for the first time, but
    mixer.music.unpause() # for each other timee

def pauseTrack():
    """
    Function to pause a track.
    """
    mixer.music.pause()


# while True: <-------- infinite loop to run our program
#
#
# (10/30)
# https://www.educative.io/answers/how-to-play-an-audio-file-in-pygame
# learn basic load, play, pause mixer functions
#
# (11/3) 
# yesterday in clas we learned how to make an object. we can use this for
# our movies. refer to Lab 10 for notes.